In [1]:
#%%timeit
#%%timeit -n 1 -r 1 # time cost for 1 run with 1 loop

############################################################### Packages
import numpy as np
import numpy.random as npr
from scipy.stats import levy_stable  
from sklearn.covariance import empirical_covariance
import matplotlib.pyplot as plt
#import random
import math 
import pandas as pd
import scipy.special
#from scipy.stats import rankdata
import numba
import timeit
import time
import warnings
import psutil

############################################################### Miscellenous

warnings.filterwarnings('ignore')
npr.seed(0)

############################################################### Section 1: Generating N*n*d real matrix as data sample for different models


######## Inductive model exhibing pairwise independence but not triplewise independence

@numba.njit(parallel=True, fastmath=True)
def inductive_model_iterated_numba(initializing_normal_sample,N,n,d):
    X=np.empty((N,n,d))
    for k in numba.prange(N):
        for i in numba.prange(n):
            X[k,i,0] = initializing_normal_sample[k,i,0]
            X[k,i,1] = initializing_normal_sample[k,i,1]
            for j in numba.prange(2,d):
                if X[k,i,j-1]+X[k,i,j-2] > 1:
                     X[k,i,j] =X[k,i,j-1]+X[k,i,j-2]-1
                else:
                    X[k,i,j] =X[k,i,j-1]+X[k,i,j-2]
    return X

######## d-variate Gumbel distribution. Algorithm by Marshall-Olkin - @numba.njit
######## https://cran.r-project.org/web/packages/copula/vignettes/nacopula-pkg.pdf

def Stable_distrib_data(N,n,t):
    if t == 1:
        V=levy_stable.rvs(1, 1, loc=1, scale=np.cos(math.pi/(2)), size=(N,n,1), random_state=None)
    else:
        V=levy_stable.rvs(1/t, 1, loc=0, scale=np.cos(math.pi/(2*t))**t, size=(N,n,1), random_state=None)
    return V

@numba.njit(parallel=True, fastmath=True)
def Gumbel_Marshall_Olkin_iterated_numba(V,d,t):
    n = V.shape[1]
    X=npr.exponential(1,size=(N,n,d))
    G=np.empty_like(X)
    for k in numba.prange(N):
        for i in numba.prange(n):
            for j in numba.prange(d):
                G[k,i,j] = np.exp(  -(X[k,i,j]/V[k,i,0]**(1/t)  ))
    return G
                                                        
######## Geisser-Mantel Model - @numba.njit

@numba.njit(parallel=True, fastmath=True)
def my_mean_numba(a):
    out = np.empty((1,a.shape[1]))
    for j in numba.prange(a.shape[1]):
        out[0,j] = np.sum(a[:,j])/a.shape[0]
    return out

@numba.njit(parallel=True, fastmath=True)
def empirical_cov_numba(G): 
    d=G.shape[1]
    n=G.shape[0]
    M=np.empty((1,d)) 
    M[0,:]=my_mean_numba(G)[0,:] # same as M[0,:]=np.mean(G,axis=0)
    I = np.ones((n,1))
    out=np.dot( np.transpose(G-np.dot(I,M)), G-np.dot(I,M) )
    return (1/(n))*out # n or n-1 ? the case n matches with scikit-learn empirical_covariance function

@numba.njit(parallel=True, fastmath=True)
def pairwise_corr_numba_numba(G,p,m): # G is a (p+m)*p real matrix
    U = empirical_cov_numba(G) # d*d matrix
    T = np.tril(U,-1) # set to 0 the upper triangular part of the square matrix U, including the diagonal
    T=T.flatten() # flatten the matrix to a row vector
    A = np.nonzero(T) # mask of indices giving nonzero values of T
    out = T[A] # 1*d array of shape (d,) 
    return np.reshape(out, (1,int(p*(p-1)/2))) # reshaping to get an array (1,d) 

@numba.njit(parallel=False, fastmath=True) # Cannot set parallel=True because of issues with slicing
def Geisser_Mantel_numba(G_n,n,p,m): 
    X = np.empty( (n,int(p*(p-1)/2)))
    for k in numba.prange(n):
        X[k,:] = pairwise_corr_numba_numba(G_n[k,:],p,m)
    return X

@numba.njit(parallel=False, fastmath=True)
def Geisser_Mantel_iterated_numba(G_N,N,n,p,m): # d=p*(p-1)/2 
    d = int(p*(p-1)/2)
    X = np.empty((N,n,d))
    for k in numba.prange(N):
        X[k,:,:] = Geisser_Mantel_numba(G_N[k,:,:,:],n,p,m)
    return X


######## Elliptical model: Gaussian vector with determined Kendall's tau L^2 norm - no njit

@numba.jit(parallel=True, fastmath=True) 
def iterated_gaussian_vector_sample(N,n,d,tau):
    p = math.sin((math.pi/2)*math.sqrt((2*tau)/(d*(d-1))))
    cov = p*np.ones((d,d))
    np.fill_diagonal(cov, 1)
    return npr.multivariate_normal(np.zeros(d),cov,size=(N,n))

####### Banded model with different band structure

@numba.njit(parallel=True, fastmath=True) 
def cov_matrix_banded(d,h_cov,rho):
    cov = 0.25*np.ones((d,d))
    np.fill_diagonal(cov, 1)
    for i in numba.prange(d):
        for j in numba.prange(d):
            if i>=h_cov+j or j>=h_cov+i:
                cov[i,j]=0
    return cov        

@numba.njit(parallel=True, fastmath=True) 
def cov_matrix_ar1(d,rho):
    cov = np.ones((d,d))
    for i in numba.prange(d):
        for j in numba.prange(d):
            cov[i,j] = rho**(np.absolute(i-j))
    return cov 

def cov_matrix_block(d,rho):
    d2=np.floor(d/5)
    A=np.identity(int(np.floor(d/5)))
    B=rho*np.ones((5,5))
    np.fill_diagonal(B, 1)
    return np.kron(A,B)

def iterated_banded_gaussian_sample(N,n,d,cov_matrix): # e.g., cov_matrix = cov_matrix_banded(d,h_cov)
    return npr.multivariate_normal(np.zeros(d),cov_matrix,size=(N,n))

def iterated_nonlinear_gaussian_sample(N,n,d):
    G=npr.multivariate_normal(np.zeros(int(np.floor(d/5))),np.identity(int(np.floor(d/5))),size=(N,n))
    G1=np.append(G, np.sin(2*np.pi*G), axis=2)
    G2=np.append(G1, np.cos(2*np.pi*G), axis=2)
    G3=np.append(G2, np.sin(4*np.pi*G), axis=2)
    out=np.append(G3, np.cos(4*np.pi*G), axis=2)
    return out

######## Truncated Romano-Siegel Model, Section 4.2 of Genest and Rémillard (2004) - pairwise indep but not jointly indep

def Romano_Siegel_iterated_data(N,n):
    Z = npr.normal(0,1,size=(N,n,5))
    for k in range(N):
        for i in range(n):
            Z[k,i,0]= np.absolute(Z[k,i,0])*np.sign(Z[k,i,1]*Z[k,i,2])
            Z[k,i,4]= Z[k,i,3]/2 + np.sqrt(3)*Z[k,i,4]/2
    return Z

@numba.njit(parallel=True, fastmath=True)
def Romano_Siegel_Numba_iterated_data(g,N,n): #g = npr.normal(0,1,size=(N,n,5))
    for k in numba.prange(N):
        for i in numba.prange(n):
            g[k,i,0]= np.absolute(g[k,i,0])*np.sign(g[k,i,1]*g[k,i,2])
            g[k,i,4]= g[k,i,3]/2 + np.sqrt(3)*g[k,i,4]/2
    return g

@numba.njit(parallel=True, fastmath=True)
def Trunc_Romano_Siegel_Numba_iterated_data(g,N,n): #g = npr.normal(0,1,size=(N,n,3))
    for k in numba.prange(N):
        for i in numba.prange(n):
            g[k,i,0]= np.absolute(g[k,i,0])*np.sign(g[k,i,1]*g[k,i,2])
    return g

@numba.njit(parallel=True, fastmath=True)
def Trunc_d_Romano_Siegel_Numba_iterated_data(g,N,n,p): #g = npr.normal(0,1,size=(N,n,3*p))
    for k in numba.prange(N):
        for i in numba.prange(n):
            for j in numba.prange(p):
                g[k,i,3*j+0]= np.absolute(g[k,i,3*j+0])*np.sign(g[k,i,3*j+1]*g[k,i,3*j+2])
    return g

############################################################### Section 2: Computing the statistics - @numba.njit

@numba.njit(parallel=True, fastmath=True)
def rank(U):
    R = np.empty_like(U)
    for j in numba.prange(U.shape[1]):
        R[:, j] = np.argsort(np.argsort(U[:, j]))+1
    return R

@numba.njit(parallel=True, fastmath=True)
def I_pij(U): # Giving a data sample U of size n*d, return the 3D-array of the elementary block I^{(p)}_{i,j}
    d = U.shape[1]
    n = U.shape[0]
    I = np.empty((d,n,n))
    R = rank(U)
    for p in numba.prange(0,d):
        for i in numba.prange(0,n):
            for j in numba.prange(0,n):
                I[p,i,j] = 1/3 + 1/(6*n) + (R[i,p]*(R[i,p]-1))/(2*n*(n+1)) + (R[j,p]*(R[j,p]-1))/(2*n*(n+1)) - max(R[i,p],R[j,p])/(n+1)
    return I

@numba.njit(parallel=True, fastmath=True)
def raw_statistics_234(U): # U data sample of size n*d
    S_2 = 0
    S_3 = 0
    S_4 = 0
    I = I_pij(U)
    d = I.shape[0]
    n = I.shape[1]
    out = np.empty(3)
    for p in numba.prange(0,d):
        for q in numba.prange(0,p):
            S_2 += (1/n)*np.sum(np.multiply(I[p,:,:],I[q,:,:]))- 1/36 + 1/(36*n)
            for r in numba.prange(0,q): 
                S_3 += (1/n)*np.sum(np.multiply(np.multiply(I[p,:,:],I[q,:,:]),I[r,:,:])) - 1/(108*n**2) + 1/(72*n) - 1/216
                # S_3 += (1/n)*np.sum(np.multiply(np.multiply(I[p,:,:],I[q,:,:]),I[r,:,:])) - (n-1)*(n-2)/(216*n*n)
                for s in numba.prange(0,r):
                        S_4 += (1/n)*np.sum(np.multiply(np.multiply(np.multiply(I[p,:,:],I[q,:,:]),I[r,:,:]),I[s,:,:]))  - ((n - 1)*(n**2 - 3*n + 3))/(1296*n**3)
    out[0] = S_2
    out[1] = S_3
    out[2] = S_4
    return out # np.array of shape (3,) representing (S2,S3,S4)
                       
def scaling_the_statistics(X,n,d,choice_scaling): # X=raw_statistics_234(U)
    out=np.empty(5)
    binom_2 = scipy.special.binom(d,2)
    binom_3 = scipy.special.binom(d,3)
    binom_4 = scipy.special.binom(d,4)
    variance_2 = ((n-2)*(n-2)*(n-1)*(8*n+1))/(32400*n*n*(n+1)*(n+1))
    scaling_finite_2 = 1/math.sqrt(variance_2*binom_2)
    scaling_2 = 90/math.sqrt(2*binom_2)
    variance_3 = (n-2)*(n-1)*(16*n - 96 + 359/n - 269/n**2 - 963/n**3 - 370/n**4)/(5832000*(n+1)**3) 
    scaling_finite_3 = 1/math.sqrt(variance_3*binom_3)
    scaling_3 = 90*math.sqrt(90)/math.sqrt(2*binom_3)
    scaling_4 = (90**2)/math.sqrt(2*binom_4)
    scaling_finite_4 = scaling_4
    if choice_scaling == 1: #finite variance scaling
        out[0] = scaling_finite_2*X[0]
        out[1] = scaling_finite_3*X[1]
        out[2] = scaling_finite_4*X[3]
        out[3] = out[0]+out[1]
        out[4] = out[0]+out[1]+out[2]
    else : #theoretical/asymptotic scaling
        out[0] = scaling_2*X[0]
        out[1] = scaling_3*X[1]
        out[2] = scaling_4*X[2]
        out[3] = out[0]+out[1]
        out[4] = out[0]+out[1]+out[2]
    return out # np.array of length 5 representing (S2, S3, S4, T3, T4)-rescaled
    
############################################################### Section 3: Output

########################################## Iterated Raw Statistics 
# without the \sqrt{2} and \sqrt{3} for T2 and T2

@numba.njit(parallel=True, fastmath=True)
def iterated_raw_statistics_234(iterated_data_sample): # iterated_data_sample is a (N,n,d)-array
    N = iterated_data_sample.shape[0]
    n = iterated_data_sample.shape[1]
    d = iterated_data_sample.shape[2]
    X = np.empty((N,3))
    U = np.empty((n,d))
    for k in numba.prange(N):
        U = iterated_data_sample[k,:,:]
        X[k,:] = raw_statistics_234(U)
    return X # np.array of shape (N,3) with column: (S2, S3, S4)

@numba.njit(parallel=True, fastmath=True)
def iterated_rescaled_statistics_234(X,n,binom_2,binom_3,binom_4,choice_scaling): #X is N*3 np.array
    N=X.shape[0]
    out=np.empty((N,5))
    variance_2 = ((n-2)*(n-2)*(n-1)*(8*n+1))/(32400*n*n*(n+1)*(n+1))
    scaling_finite_2 = 1/math.sqrt(variance_2*binom_2)
    scaling_2 = 90/math.sqrt(2*binom_2)
    variance_3 = (n-2)*(n-1)*(16*n - 96 + 359/n - 269/n**2 - 963/n**3 - 370/n**4)/(5832000*(n+1)**3) 
    scaling_finite_3 = 1/math.sqrt(variance_3*binom_3)
    scaling_3 = 90*math.sqrt(90)/math.sqrt(2*binom_3)
    scaling_4 = (90**2)/math.sqrt(2*binom_4)
    scaling_finite_4 = scaling_4
    for k in numba.prange(N):
        if choice_scaling == 1: #finite variance scaling
            out[k,0] = scaling_finite_2*X[k,0]
            out[k,1] = scaling_finite_3*X[k,1]
            out[k,2] = scaling_finite_4*X[k,3]
            out[k,3] = out[k,0]+out[k,1]
            out[k,4] = out[k,0]+out[k,1]+out[k,2]
        else : #theoretical/asymptotic scaling
            out[k,0] = scaling_2*X[k,0]
            out[k,1] = scaling_3*X[k,1]
            out[k,2] = scaling_4*X[k,2]
            out[k,3] = out[k,0]+out[k,1]
            out[k,4] = out[k,0]+out[k,1]+out[k,2]
    return out # np.array of shape (N,5) with column: (S2, S3, S4, T3, T4)-rescaled

######### Power 

def power(A,x):
    B=A[A>x]
    return B.size/A.shape[0]

def Gaussian_power_for_each_columns_234(A):
    X=np.empty((1,5))
    for i in range(3):
        X[0,i]=power(A[:,i],1.645) # 95-percentile of a N(0,1)
    X[0,3]=power((1/np.sqrt(2))*(A[:,3]),1.645) 
    X[0,4]=power((1/np.sqrt(3))*(A[:,4]),1.645) 
    return X

def Gaussian_power_for_each_columns_23_new(A):
    X=np.empty((1,3))
    for i in range(3):
        X[0,i]=power(A[:,i],1.645) # 95-percentile of a N(0,1)
    return X

def CF_power_for_each_columns_234(A,d): # Cornish-Fisher
    X=np.empty((1,5))
    if d==4:
        X[0,0]=power(A[:,0],1.858792) 
        X[0,1]=power(A[:,1],1.867722) 
        X[0,2]=power(A[:,2],1.938571) 
        X[0,3]=power((1/np.sqrt(2))*(A[:,3]),1.813631) 
        X[0,4]=power((1/np.sqrt(3))*(A[:,4]),1.821249)
    if d==8:
        X[0,0]=power(A[:,0],1.758067) 
        X[0,1]=power(A[:,1],1.719549) 
        X[0,2]=power(A[:,2],1.712121) 
        X[0,3]=power((1/np.sqrt(2))*(A[:,3]),1.712876) 
        X[0,4]=power((1/np.sqrt(3))*(A[:,4]),1.695521)
    if d==16:
        X[0,0]=power(A[:,0],1.702343) 
        X[0,1]=power(A[:,1],1.669482) 
        X[0,2]=power(A[:,2],1.658623) 
        X[0,3]=power((1/np.sqrt(2))*(A[:,3]),1.674184) 
        X[0,4]=power((1/np.sqrt(3))*(A[:,4]),1.663556)
    if d==32:
        X[0,0]=power(A[:,0],1.67375) 
        X[0,1]=power(A[:,1],1.653226) 
        X[0,2]=power(A[:,2],1.647974) 
        X[0,3]=power((1/np.sqrt(2))*(A[:,3]),1.658101) 
        X[0,4]=power((1/np.sqrt(3))*(A[:,4]),1.652684)
    if d==64:
        X[0,0]=power(A[:,0],1.659332) 
        X[0,1]=power(A[:,1],1.647753) 
        X[0,2]=power(A[:,2],1.645597) 
        X[0,3]=power((1/np.sqrt(2))*(A[:,3]),1.651016) 
        X[0,4]=power((1/np.sqrt(3))*(A[:,4]),1.648355)
    if d==128:
        X[0,0]=power(A[:,0],1.652099) 
        X[0,1]=power(A[:,1],1.645868) 
        X[0,2]=power(A[:,2],1.645035) 
        X[0,3]=power((1/np.sqrt(2))*(A[:,3]),1.647779) 
        X[0,4]=power((1/np.sqrt(3))*(A[:,4]),1.646482)
    if d==256:
        X[0,0]=power(A[:,0],1.648478) 
        X[0,1]=power(A[:,1],1.64521) 
        X[0,2]=power(A[:,2],1.644898) 
        X[0,3]=power((1/np.sqrt(2))*(A[:,3]),1.646262) 
        X[0,4]=power((1/np.sqrt(3))*(A[:,4]),1.645629)
    return X

def Romano_Siegel_index(x): 
    y=0
    if x==4:
        y=3
    if x==8:
        y=6
    if x==16:
        y=15
    if x==32:
        y=30
    if x==64:
        y=63
    if x==128:
        y=126
    if x==256:
        y=255
    return y 

def Geisser_Mantel_index(x):
    y=0
    if x==4:
        y=3
    if x==8:
        y=6
    if x==16:
        y=10
    if x==32:
        y=28
    if x==64:
        y=55
    if x==128:
        y=120
    if x==256:
        y=231
    return y 

############ All-in-one: Sampling iterated data + Computing iterated statistics (raw and rescaled) + Tabulars

def all_data_stat_iid234(N):
    for n in [16,32,64,128]:
        for d in [4,8,16,32,64,128,256]:
            binom_2 = scipy.special.binom(d,2)
            binom_3 = scipy.special.binom(d,3)
            binom_4 = scipy.special.binom(d,4)
            iterated_data_sample = npr.exponential(1,size=(N,n,d))
            X=iterated_raw_statistics_234(iterated_data_sample)
            pd.DataFrame(X).to_csv("Raw_iid_234{}.csv".format((N,n,d)), header=False, index=False)
            for choice_scaling in [0,1]:
                A=iterated_rescaled_statistics_234(X,n,binom_2,binom_3,binom_4,choice_scaling)
                pd.DataFrame(A).to_csv("GR_234_iid{}.csv".format((N,n,d,choice_scaling)), header=False, index=False)
    return 'done!'

def total_power_iid_234(N,choice_scaling,CF):
    X=np.empty((4,5,7))
    mean = 1
    for j in range(5):
        for n in [16, 32, 64, 128]:
            for d in [4,8,16,32,64,128,256]:
                d2=int(math.log(d)/math.log(2))-2
                n2=int(math.log(n)/math.log(2))-4
                A=pd.read_csv('GR_234_iid{}.csv'.format((N,n,d,choice_scaling)),header=None)
                if CF == 1:
                    X[n2,j,d2]=CF_power_for_each_columns_234(A.values,d)[0,j]
                else:
                    X[n2,j,d2]=Gaussian_power_for_each_columns_234(A.values)[0,j]
    A_1=X[0,:,:]
    B=X[1,:,:]
    C=X[2,:,:]
    D=X[3,:,:]
    out = np.concatenate((np.concatenate((np.concatenate((A_1,B),axis=0),C),axis=0),D),axis=0)
    if CF == 1:
        pd.DataFrame(out).to_csv("Tabular_CF_iid_234{}.csv".format((N,choice_scaling)), header=False, index=False)
    else:
        pd.DataFrame(out).to_csv("Tabular_iid_234{}.csv".format((N,choice_scaling)), header=False, index=False)
    return 100*out

def all_data_stat_Gaussian_banded234(N,h_cov,banded_model):
    for n in [60,100]:
        for d in [50,100,200,400]:
            binom_2 = scipy.special.binom(d,2)
            binom_3 = scipy.special.binom(d,3)
            binom_4 = scipy.special.binom(d,4)
            if banded_model == 0:
                cov_matrixx = cov_matrix_banded(d,h_cov,rho)
            elif banded_model==1:
                cov_matrixx = cov_matrix_ar1(d,rho)
            else:
                cov_matrixx = cov_matrix_block(d,rho)                
            iterated_data_sample=iterated_gaussian_vector_sample(N,n,d,cov_matrixx)
            X=iterated_raw_statistics_234(iterated_data_sample)
            pd.DataFrame(X).to_csv("Raw_Statistics_Gaussian_Banded_234{}.csv".format((N,n,d,h_cov,rho,banded_model)), header=False, index=False)            
            for choice_scaling in [0,1]:
                A=iterated_rescaled_statistics_234(X,n,binom_2,binom_3,binom_4,choice_scaling)
                if banded_model == 0:
                    pd.DataFrame(A).to_csv("GR234_Band{}.csv".format((N,n,d,h_cov,rho,choice_scaling)),header=None)
                elif banded_model==1:
                    pd.DataFrame(A).to_csv("GR234_AR(1){}.csv".format((N,n,d,rho,choice_scaling)),header=None)
                else:
                    pd.DataFrame(A).to_csv("GR234_Block{}.csv".format((N,n,d,rho,choice_scaling)),header=None)    
    return 'done!'

def total_power_Gaussian_Banded_23(N,h_cov,rho,banded_model,choice_scaling):
    X=np.empty((2,3,4))
    mean = 1
    for j in range(3):
        for n in [60,100]:
            for d in [50,100,200,400]:
                if n==60:
                    n2=0
                elif n==100:
                    n2=1
                if d==50:
                    d2=0
                elif d==100:
                    d2=1
                elif d==200:
                    d2=2
                elif d==400:
                    d2=3
                if banded_model == 0:
                    A=pd.read_csv("GR234_Band{}.csv".format((N,n,d,h_cov,rho,choice_scaling)),header=None)
                elif banded_model==1:
                    A=pd.read_csv("GR234_AR(1){}.csv".format((N,n,d,rho,choice_scaling)),header=None)
                else:
                    A=pd.read_csv("GR234_Block{}.csv".format((N,n,d,rho,choice_scaling)),header=None)                    
                Y=A.values
                X[n2,j,d2]=Gaussian_power_for_each_columns_23_new(Y)[0,j]
    A_1=X[0,:,:]
    B=X[1,:,:]
    out = np.concatenate((A_1,B),axis=0)
    if banded_model == 0:
        pd.DataFrame(out).to_csv("Tabular_Band{}.csv".format((N,h_cov,rho,choice_scaling)), header=False, index=False)
    elif banded_model==1:
        pd.DataFrame(out).to_csv("Tabular_AR(1){}.csv".format((N,rho,choice_scaling)), header=False, index=False)
    else:
        pd.DataFrame(out).to_csv("Tabular_Block_{}.csv".format((N,rho,choice_scaling)), header=False, index=False)        
    return 100*out 

def all_data_stat_inductive234(N):
    for n in [16,32,64,128]:
        for d in [4,8,16,32,64,128,256]:
            binom_2 = scipy.special.binom(d,2)
            binom_3 = scipy.special.binom(d,3)
            binom_4 = scipy.special.binom(d,4)
            initializing_normal_sample=npr.uniform(0,1,size=(N,n,2))
            iterated_data_sample = inductive_model_iterated_numba(initializing_normal_sample,N,n,d)
            X=iterated_raw_statistics_234(iterated_data_sample)
            pd.DataFrame(X).to_csv("Raw_inductive_234{}.csv".format((N,n,d)), header=False, index=False)
            for choice_scaling in [0,1]:
                A=iterated_rescaled_statistics_234(X,n,binom_2,binom_3,binom_4,choice_scaling)
                pd.DataFrame(A).to_csv("GR_234_inductive{}.csv".format((N,n,d,choice_scaling)), header=False, index=False)
    return 'done!'

def total_power_inductive_234(N,choice_scaling,CF):
    X=np.empty((4,5,7))
    for j in range(5):
        for n in [16, 32, 64, 128]:
            for d in [4,8,16,32,64,128,256]:
                d2=int(math.log(d)/math.log(2))-2
                n2=int(math.log(n)/math.log(2))-4
                A=pd.read_csv('GR_234_inductive{}.csv'.format((N,n,d,choice_scaling)),header=None)
                if CF == 1:   
                    X[n2,j,d2]=CF_power_for_each_columns_234(A.values,d)[0,j]
                else:
                    X[n2,j,d2]=Gaussian_power_for_each_columns_234(A.values)[0,j]
    A_1=X[0,:,:]
    B=X[1,:,:]
    C=X[2,:,:]
    D=X[3,:,:]
    out = np.concatenate((np.concatenate((np.concatenate((A_1,B),axis=0),C),axis=0),D),axis=0)
    if CF == 1:
        pd.DataFrame(out).to_csv("Tabular_CF_Inductive_234{}.csv".format((N,choice_scaling)), header=False, index=False)
    else:
        pd.DataFrame(out).to_csv("Tabular_Inductive_234{}.csv".format((N,choice_scaling)), header=False, index=False)
    return 100*out 

def all_data_stat_Gaussian_Vector234(N):
    for tau in [0.1,0.3,0.7]:
        for n in [16,32,64,128]:
            for d in [4,8,16,32,64,128,256]:
                binom_2 = scipy.special.binom(d,2)
                binom_3 = scipy.special.binom(d,3)
                binom_4 = scipy.special.binom(d,4)
                iterated_data_sample = iterated_gaussian_vector_sample(N,n,d,tau)
                X=iterated_raw_statistics_234(iterated_data_sample)
                pd.DataFrame(X).to_csv("Raw_Gaussian_Vector_234{}.csv".format((N,n,d,tau)), header=False, index=False)
                for choice_scaling in [0,1]:
                    A=iterated_rescaled_statistics_234(X,n,binom_2,binom_3,binom_4,choice_scaling)
                    pd.DataFrame(A).to_csv("GR_234_Gaussian_vector{}.csv".format((N,n,d,tau,choice_scaling)), header=False, index=False)
    return 'done!'

def total_power_Gaussian_vector_234(N,tau,choice_scaling,CF):
    X=np.empty((4,5,7))
    for j in range(5):
        for n in [16, 32, 64, 128]:
            for d in [4,8,16,32,64,128,256]:
                d2=int(math.log(d)/math.log(2))-2
                n2=int(math.log(n)/math.log(2))-4
                A=pd.read_csv('GR_234_Gaussian_vector{}.csv'.format((N,n,d,tau,choice_scaling)),header=None)
                if CF == 1:
                    X[n2,j,d2]=CF_power_for_each_columns_234(A.values,d)[0,j]
                else:
                    X[n2,j,d2]=Gaussian_power_for_each_columns_234(A.values)[0,j]    
    A_1=X[0,:,:]
    B=X[1,:,:]
    C=X[2,:,:]
    D=X[3,:,:]
    out = np.concatenate((np.concatenate((np.concatenate((A_1,B),axis=0),C),axis=0),D),axis=0)
    if CF == 1:
        pd.DataFrame(out).to_csv("Tabular_CF_Gaussian_Vector_234{}.csv".format((N,tau,choice_scaling)), header=False, index=False)
    else:
        pd.DataFrame(out).to_csv("Tabular_Gaussian_Vector_234{}.csv".format((N,tau,choice_scaling)), header=False, index=False)
    return 100*out   

def all_data_stat_Geisser_Mantel234(N):
    for n in [16,32,64,128]:
        for d in [4,8,16,32,64,128,256]:
            p = max(math.floor(math.sqrt(2*d)),3)
            q=int(p*(p-1)/2)
            m = math.floor(p/2)
            G = npr.multivariate_normal(np.zeros(p),np.eye(p),size=p+m)
            G_N = npr.multivariate_normal(np.zeros(p),np.eye(p),size=(int(N),int(n),int(p+m)))
            binom_2 = scipy.special.binom(d,2)
            binom_3 = scipy.special.binom(d,3)
            binom_4 = scipy.special.binom(d,4)
            iterated_data_sample=Geisser_Mantel_iterated_numba(G_N,N,n,p,m)
            X=iterated_raw_statistics_234(iterated_data_sample)
            pd.DataFrame(X).to_csv("Raw_Geisser_Mantel_234{}.csv".format((N,n,q)), header=False, index=False)
            for choice_scaling in [0,1]:
                A=iterated_rescaled_statistics_234(X,n,binom_2,binom_3,binom_4,choice_scaling)
                pd.DataFrame(A).to_csv("GR_234_Geisser_Mantel{}.csv".format((N,n,q,choice_scaling)), header=False, index=False)
    return 'done!'

def total_power_Geisser_Mantel_234(N,choice_scaling,CF):
    X=np.empty((4,5,7))
    for j in range(5):
        for n in [16, 32, 64, 128]:
            for d in [4,8,16,32,64,128,256]: # [3,6,10,28,55,120,231]
                d2=int(math.log(d)/math.log(2))-2
                n2=int(math.log(n)/math.log(2))-4
                A=pd.read_csv('GR_234_Geisser_Mantel{}.csv'.format((N,n,Geisser_Mantel_index(d),choice_scaling)),header=None)
                if CF == 1:
                    X[n2,j,d2]=CF_power_for_each_columns_234(A.values,d)[0,j]
                else:
                    X[n2,j,d2]=Gaussian_power_for_each_columns_234(A.values)[0,j]
    A_1=X[0,:,:]
    B=X[1,:,:]
    C=X[2,:,:]
    D=X[3,:,:]
    out = np.concatenate((np.concatenate((np.concatenate((A_1,B),axis=0),C),axis=0),D),axis=0)
    if CF == 1:
        pd.DataFrame(out).to_csv("Tabular_CF_Geisser_Mantel_234{}.csv".format((N,choice_scaling)), header=False, index=False)
    else:
        pd.DataFrame(out).to_csv("Tabular_Geisser_Mantel_234{}.csv".format((N,choice_scaling)), header=False, index=False)        
    return 100*out  

def all_data_stat_Romano_Siegel234(N):
    for n in [16,32,64,128]:
        for d in [4,8,16,32,64,128,256]:
            p=int(max(math.floor(d/3),1))
            q=int(3*p)
            g = npr.normal(0,1,size=(N,n,q))
            iterated_data_sample=Trunc_d_Romano_Siegel_Numba_iterated_data(g,N,n,p)
            binom_2 = scipy.special.binom(d,2)
            binom_3 = scipy.special.binom(d,3)
            binom_4 = scipy.special.binom(d,4)
            X=iterated_raw_statistics_234(iterated_data_sample)
            pd.DataFrame(X).to_csv("Raw_Trunc_d_Romano_Siegel_234{}.csv".format((N,n,q)), header=False, index=False)
            for choice_scaling in [0,1]:
                A=iterated_rescaled_statistics_234(X,n,binom_2,binom_3,binom_4,choice_scaling)
                pd.DataFrame(A).to_csv("GR_234_Trunc_d_Romano_Siegel{}.csv".format((N,n,q,choice_scaling)), header=False, index=False)
    return 'done!'

def total_power_Romano_Siegel_234(N,choice_scaling,CF):
    X=np.empty((4,5,7))
    for j in range(5):
        for n in [16, 32, 64, 128]:
            for d in [4,8,16,32,64,128,256]: # [3,6,15,30,63,126,255]
                d2=int(math.log(d)/math.log(2))-2
                n2=int(math.log(n)/math.log(2))-4
                A=pd.read_csv('GR_234_Trunc_d_Romano_Siegel{}.csv'.format((N,n,Romano_Siegel_index(d),choice_scaling)),header=None)
                if CF == 1:
                    X[n2,j,d2]=CF_power_for_each_columns_234(A.values,d)[0,j]
                else:
                    X[n2,j,d2]=Gaussian_power_for_each_columns_234(A.values)[0,j]
    A_1=X[0,:,:]
    B=X[1,:,:]
    C=X[2,:,:]
    D=X[3,:,:]
    out = np.concatenate((np.concatenate((np.concatenate((A_1,B),axis=0),C),axis=0),D),axis=0)
    if CF == 1:
        pd.DataFrame(out).to_csv("Tabular_CF_Romano_Siegel_234{}.csv".format((N,choice_scaling)), header=False, index=False)
    else:
        pd.DataFrame(out).to_csv("Tabular_Romano_Siegel_234{}.csv".format((N,choice_scaling)), header=False, index=False)
    return 100*out  

############################################################### Section 4: Global variables

start = time.time()

#N = 500 # Number of iterations
#n = 16
#d = 16
#choice_scaling = 0 # 1 is finite variance, 0 is for asymptotic/theoretical scaling


### Pre-computations of the binomial coefficients 
#binom_2 = scipy.special.binom(d,2)
#binom_3 = scipy.special.binom(d,3)
#binom_4 = scipy.special.binom(d,4)

### Pre-computations of the normalizing sequences
#variance_2 = ((n-2)*(n-2)*(n-1)*(8*n+1))/(32400*n*n*(n+1)*(n+1))
#scaling_finite_2 = 1/math.sqrt(variance_2*binom_2)
#scaling_2 = 90/math.sqrt(2*binom_2)
#variance_3 = (n-2)*(n-1)*(16*n - 96 + 359/n - 269/n**2 - 963/n**3 - 370/n**4)/(5832000*(n+1)**3) 
#scaling_finite_3 = 1/math.sqrt(variance_3*binom_3)
#scaling_3 = 90*math.sqrt(90)/math.sqrt(2*binom_3)
#scaling_4 = (90**2)/math.sqrt(2*binom_4)
#scaling_finite_4 = scaling_4

#mean = 1

### Pre-sampling the intput data for Geisser-Mantel Numba Model
#p = max(math.floor(math.sqrt(2*d)),3)
#m = math.floor(p/2)
#G = npr.multivariate_normal(np.zeros(p),np.eye(p),size=p+m)
#G_N = npr.multivariate_normal(np.zeros(p),np.eye(p),size=(int(N),int(n),int(p+m)))
#G_n = npr.multivariate_normal(np.zeros(p),np.eye(p),size=(n,p+m))

### Pre-sampling the intput data for Marshall-Olkin Algorithm
#t = 1
#V = Stable_distrib_data(N,n,t)
                                                                    
### Pre-sampling the intput data for the inductive model
#initializing_normal_sample=npr.uniform(0,1,size=(N,n,2))

### Parameters for the Gaussian vector model
#tau = 0.1
#tau = 0.3
#tau = 0.7

###### Parameters for the Gaussian vector with banded structure (Band, AR(1), Block from Yao et al. 2018)
# h_cov = 3
# rho = 0.25

#### Bandwidth of the banded test 
#h_1=h_cov # drives (p,q)
#h_2=h_cov # drives (q,r)
#h_3=h_cov # drives (p,r)

# banded_model = 0 for Band structure Gaussian, 1 for AR(1)-structure, 2 for block structure
# banded_model = 3 is for nonlinear Gaussian vector as in Example 6.3 in Yao et al. 2018

############################################################### Section 5: Application
########################### Section 5.1: Generating iterated data samples of size N*n*d
                                  
#iterated_data_sample=Tan_iterated_data(N,n,d)
#p=int(max(math.floor(d/3),1))
#g = npr.normal(0,1,size=(N,n,3*p))
#iterated_data_sample = Trunc_d_Romano_Siegel_Numba_iterated_data(g,N,n,p)
#iterated_data_sample = npr.normal(size=(N,n,d))
#iterated_data_sample = npr.exponential(1,size=(N,n,d))
#iterated_data_sample = npr.multivariate_normal(np.zeros(d),random_cov,size=(int(N),int(n),int(d)))
#iterated_data_sample = npr.multivariate_normal(np.zeros(d),toeplitz_cov,size=((N),int(n)))
#iterated_data_sample = inductive_model_iterated_numba(initializing_normal_sample,N,n,d)
#iterated_data_sample = Gumbel_Marshall_Olkin_iterated_numba(V,d,t)
#iterated_data_sample = Geisser_Mantel_iterated_numba(G_N,N,n,p,m)
#iterated_data_sample = iterated_gaussian_vector_sample(N,n,d,tau)

#all_data_stat_iid234(N)
#all_data_stat_Romano_Siegel234(N)
#all_data_stat_Geisser_Mantel234(N)
#all_data_stat_Gaussian_Vector234(N)
#all_data_stat_inductive234(N)

########################### Section 5.3: Computing the statistics from the iterated data samples

#X=iterated_raw_statistics_234(iterated_data_sample)
#A=iterated_rescaled_statistics_234(X,n,binom_2,binom_3,binom_4,choice_scaling)
                                  
########################### Section 5.4: Displaying (histogram) and converting into Panda files the computed statistics

#A_df = pd.DataFrame(A)   
#A_df.hist()

########################### Section 5.5: Storing in Excel files the iterated raw statistics N*2 or N*3 with column (S2,S3,S4)
                                  
#pd.DataFrame(X).to_csv("Raw_Normal_iid{}.csv".format((N,n,d)), header=False, index=False)
#pd.DataFrame(X).to_csv("Raw_Inductive{}.csv".format((N,n,d)), header=False, index=False)
#pd.DataFrame(X).to_csv("Raw_Random_Cov{}.csv".format((N,n,d)), header=False, index=False)
#pd.DataFrame(X).to_csv("Raw_Toeplitz{}.csv".format((N,n,d)), header=False, index=False)
#pd.DataFrame(X).to_csv("Raw_Gumbel{}.csv".format((N,n,d,t)), header=False, index=False)
#pd.DataFrame(X).to_csv("Raw_Geisser_Mantel{}.csv".format((N,n,p,m)), header=False, index=False)
#pd.DataFrame(X).to_csv("Raw_Gaussian_Vector{}.csv".format((N,n,d,tau)), header=False, index=False)
#pd.DataFrame(X).to_csv("Raw_Exponential_iid{}.csv".format((N,n,d)), header=False, index=False)

########################### Section 5.6: Storing in Excel files the final statistics (rescaled raw)
                                  
#pd.DataFrame(A).to_csv("GR_234_inductive{}.csv".format((N,n,d,choice_scaling)), header=False, index=False)
#pd.DataFrame(A).to_csv("GR_234_Gumbel_Marshall_Olkin{}.csv".format((N,n,d,t,choice_scaling)), header=False, index=False)
#pd.DataFrame(A).to_csv("GR_234_Geisser_Mantel{}.csv".format((N,n,d,choice_scaling)), header=False, index=False)
#pd.DataFrame(A).to_csv("GR_234_Gaussian_vector{}.csv".format((N,n,d,tau,choice_scaling)), header=False, index=False)
#pd.DataFrame(A).to_csv("GR_234_Exponential_iid{}.csv".format((N,n,d,mean,choice_scaling)), header=False, index=False)
#pd.DataFrame(A).to_csv("GR_234_Random_Cov_Gauss{}.csv".format((N,n,d,choice_scaling)), header=False, index=False)
#pd.DataFrame(A).to_csv("GR_234_Toeplitz_Cov_Gauss{}.csv".format((N,n,d,choice_scaling)), header=False, index=False)
#pd.DataFrame(A).to_csv("GR_234_Tan{}.csv".format((N,n,d,choice_scaling)), header=False, index=False)
#pd.DataFrame(A).to_csv("GR_234_Trunc_d_Romano_Siegel{}.csv".format((N,n,d,choice_scaling)), header=False, index=False)

end = time.time()
print("Time: ", end-start)

Time:  1.7642974853515625e-05


In [2]:
#N = 500
#choice_scaling = 0
#CF = 1
#tau1 = 0.1
#tau2 = 0.3
#tau3 = 0.7
#total_power_iid_234(N,choice_scaling,CF)
#total_power_Gaussian_vector_234(N,tau1,choice_scaling,CF)
#total_power_Gaussian_vector_234(N,tau2,choice_scaling,CF)
#total_power_Gaussian_vector_234(N,tau3,choice_scaling,CF)
#total_power_inductive_234(N,choice_scaling,CF)
#total_power_Geisser_Mantel_234(N,choice_scaling,CF)
#total_power_Romano_Siegel_234(N,choice_scaling,CF)

In [3]:
############# Banded Statistics

@numba.njit(parallel=True, fastmath=True)
def banded_raw_statistics_2(U,h): # U data sample of size n*d 
    S_2 = 0
    S_3 = 0
    S_4 = 0
    I = I_pij(U)
    d = I.shape[0]
    n = I.shape[1]
    out = np.empty(2)
    for p in numba.prange(0,d):
        for q in numba.prange(0,p):
            if p>=h+q: #or q>=h+p:
                S_2 += (1/n)*np.sum(np.multiply(I[p,:,:],I[q,:,:]))- 1/36 + 1/(36*n)
    return S_2

@numba.njit(parallel=True, fastmath=True)
def banded_raw_statistics_3(U,h_1,h_2,h_3): # U data sample of size n*d 
    # h_1 drives the distance between (p,q), h_2 for (q,r) and h_3 for (p,r)
    S_2 = 0
    S_3 = 0
    S_4 = 0
    I = I_pij(U)
    d = I.shape[0]
    n = I.shape[1]
    out = np.empty(2)
    for p in numba.prange(0,d):
        for q in numba.prange(0,p):
            if p>=h_1+q: #or q>=h_1+p:
                for r in numba.prange(0,q): 
                    if q>=h_2+r and p>=h_3+r:
                        S_3 += (1/n)*np.sum(np.multiply(np.multiply(I[p,:,:],I[q,:,:]),I[r,:,:])) - 1/(108*n**2) + 1/(72*n) - 1/216 
                        # S_3 += (1/n)*np.sum(np.multiply(np.multiply(I[p,:,:],I[q,:,:]),I[r,:,:])) - (n-1)*(n-2)/(216*n*n)

    return S_3 

@numba.njit(parallel=True, fastmath=True)
def banded_raw_statistics_23(U,h_1,h_2,h_3): # U data sample of size n*d 
    # h_1 drives the distance between (p,q), h_2 for (q,r) and h_3 for (p,r)
    S_2 = 0
    S_3 = 0
    S_4 = 0
    I = I_pij(U)
    d = I.shape[0]
    n = I.shape[1]
    out = np.empty(2)
    for p in numba.prange(0,d):
        for q in numba.prange(0,p):
            if p>=h_1+q: #or q>=h_1+p:
                S_2 += (1/n)*np.sum(np.multiply(I[p,:,:],I[q,:,:]))- 1/36 + 1/(36*n)
                for r in numba.prange(0,q): 
                    if q>=h_2+r and p>=h_3+r:
                        S_3 += (1/n)*np.sum(np.multiply(np.multiply(I[p,:,:],I[q,:,:]),I[r,:,:])) - 1/(108*n**2) + 1/(72*n) - 1/216 
                        # S_3 += (1/n)*np.sum(np.multiply(np.multiply(I[p,:,:],I[q,:,:]),I[r,:,:])) - (n-1)*(n-2)/(216*n*n)
    out[0] = S_2
    out[1] = S_3
    return out # np.array of shape (2,) representing (S2,S3)

@numba.njit(parallel=True, fastmath=True)
def banded_cardinal_23(d,h_1,h_2,h_3): # computes the cardinal of all ordered integer pairs and triples with
    # bandwidth condition driven by (h_1,h_2,h_3)
    out2 = np.zeros((d,d))
    out3 = np.zeros((d,d,d))
    out = np.zeros(2)
    for p in numba.prange(0,d):
        for q in numba.prange(0,p):
            if p>=h_1+q: #or q>=h_1+p:
                out2[p,q] = 1
                for r in numba.prange(0,q): 
                    if q>=h_2+r and p>=h_3+r:
                        out3[p,q,r] = 1
    out[0]=np.sum(out2) # cardinal of all ordered pairs with distance between components driven by h[0] 
    out[1]=np.sum(out3) # cardinal of all ordered triples with pairwise distance condition driven by h
    return out 

def banded_rescaled_statistics_2(X,n,d,h,choice_scaling): # X=banded_raw_statistics_2(U,h)
    # U data sample of shape (n,d)
    binom_2 = banded_cardinal_23(d,h,h,h)[0]
    variance_2 = ((n-2)*(n-2)*(n-1)*(8*n+1))/(32400*n*n*(n+1)*(n+1))
    scaling_finite_2 = 1/math.sqrt(variance_2*binom_2)
    scaling_2 = 90/math.sqrt(2*binom_2)
    if choice_scaling == 1: #finite variance scaling
        out = scaling_finite_2*X
    else : #theoretical/asymptotic scaling
        out = scaling_2*X
    return out # (S2)-rescaled according to the 'banded scaling' 

def banded_rescaled_statistics_3(X,n,d,h_1,h_2,h_3,choice_scaling): # X=banded_raw_statistics_3(U,h_1,h_2,h_3)
    # U data sample of shape (n,d)
    binom_3 = banded_cardinal_23(d,h_1,h_2,h_3)[1]
    variance_3 = (n-2)*(n-1)*(16*n - 96 + 359/n - 269/n**2 - 963/n**3 - 370/n**4)/(5832000*(n+1)**3) 
    scaling_finite_3 = 1/math.sqrt(variance_3*binom_3)
    scaling_3 = 90*math.sqrt(90)/math.sqrt(2*binom_3)
    if choice_scaling == 1: #finite variance scaling
        out = scaling_finite_3*X
    else : #theoretical/asymptotic scaling
        out = scaling_3*X
    return out # (S3)-rescaled according to the 'banded scaling' 

def banded_rescaled_statistics_23(X,n,d,h_1,h_2,h_3,choice_scaling): # X=banded_raw_statistics_23(U,h_1,h_2,h_3)
    # U data sample of shape (n,d)
    out=np.empty(3)
    cardinal = banded_cardinal_23(d,h_1,h_2,h_3)
    binom_2 = cardinal[0]
    binom_3 = cardinal[1]
    variance_2 = ((n-2)*(n-2)*(n-1)*(8*n+1))/(32400*n*n*(n+1)*(n+1))
    scaling_finite_2 = 1/math.sqrt(variance_2*binom_2)
    scaling_2 = 90/math.sqrt(2*binom_2)
    variance_3 = (n-2)*(n-1)*(16*n - 96 + 359/n - 269/n**2 - 963/n**3 - 370/n**4)/(5832000*(n+1)**3) 
    scaling_finite_3 = 1/math.sqrt(variance_3*binom_3)
    scaling_3 = 90*math.sqrt(90)/math.sqrt(2*binom_3)
    if choice_scaling == 1: #finite variance scaling
        out[0] = scaling_finite_2*X[0]
        out[1] = scaling_finite_3*X[1]
        out[2] = (1/np.sqrt(2))*(out[0]+out[1])
    else : #theoretical/asymptotic scaling
        out[0] = scaling_2*X[0]
        out[1] = scaling_3*X[1]
        out[2] = (1/np.sqrt(2))*(out[0]+out[1])
    return out # np.array of length 3 representing (S2, S3, T3)-rescaled according to the 'banded scaling' 

@numba.njit(parallel=True, fastmath=True)
def iterated_banded_raw_statistics_23(iterated_data_sample,h_1,h_2,h_3): # iterated_data_sample is a (N,n,d)-array
    N = iterated_data_sample.shape[0]
    n = iterated_data_sample.shape[1]
    d = iterated_data_sample.shape[2]
    X = np.empty((N,2))
    U = np.empty((n,d))
    for k in numba.prange(N):
        U = iterated_data_sample[k,:,:]
        X[k,:] = banded_raw_statistics_23(U,h_1,h_2,h_3)
    return X # np.array of shape (N,2) with column: (S2, S3)

def all_data_banded_stat_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling): # h_cov = 3 #rho = 0.25
    # banded_model = 0 for Band structure Gaussian, 1 for AR(1)-structure, 2 for block structure
    # banded_model = 3 is for nonlinear Gaussian vector as in Example 6.3 in Yao et al. 2018
    for n in [60,100]:
        for d in [50,100,200,400]:
            if banded_model == 0:
                cov_matrixx = cov_matrix_banded(d,h_cov,rho)
            elif banded_model==1:
                cov_matrixx = cov_matrix_ar1(d,rho)
            else:
                cov_matrixx = cov_matrix_block(d,rho)                
            iterated_data_sample=iterated_banded_gaussian_sample(N,n,d,cov_matrixx)
            if banded_model == 0:
                iterated_data_sample2=iterated_data_sample**(1/3)
                iterated_data_sample3=iterated_data_sample**(3)
            if banded_model == 3:
                iterated_data_sample4=iterated_nonlinear_gaussian_sample(N,n,d)
            X=iterated_banded_raw_statistics_23(iterated_data_sample,h_1,h_2,h_3)
            if banded_model == 0:
                X2=iterated_banded_raw_statistics_23(iterated_data_sample2,h_1,h_2,h_3)
                X3=iterated_banded_raw_statistics_23(iterated_data_sample3,h_1,h_2,h_3)
            if banded_model == 3:
                X4=iterated_banded_raw_statistics_23(iterated_data_sample4,h_1,h_2,h_3)
            pd.DataFrame(X).to_csv("Raw_Banded_Statistics_23{}.csv".format((N,n,d,h_cov,rho,banded_model)), header=False, index=False)
            if banded_model == 0:    
                pd.DataFrame(X2).to_csv("Raw_Banded_Root_Statistics_23{}.csv".format((N,n,d,h_cov,rho,banded_model)), header=False, index=False)  
                pd.DataFrame(X3).to_csv("Raw_Banded_InvRoot_Statistics_23{}.csv".format((N,n,d,h_cov,rho,banded_model)), header=False, index=False)            
            if banded_model == 3:            
                pd.DataFrame(X4).to_csv("Raw_Banded_Nonlinear_Statistics_23{}.csv".format((N,n,d,h_cov)), header=False, index=False)            
            A=np.empty((N,3))
            A2=np.empty((N,3))
            A3=np.empty((N,3))
            A4=np.empty((N,3))
            cardinal = banded_cardinal_23(d,h_cov,h_cov,h_cov)
            binom_2 = cardinal[0]
            binom_3 = cardinal[1]
            variance_2 = ((n-2)*(n-2)*(n-1)*(8*n+1))/(32400*n*n*(n+1)*(n+1))
            scaling_finite_2 = 1/math.sqrt(variance_2*binom_2)
            scaling_2 = 90/math.sqrt(2*binom_2)
            variance_3 = (n-2)*(n-1)*(16*n - 96 + 359/n - 269/n**2 - 963/n**3 - 370/n**4)/(5832000*(n+1)**3) 
            scaling_finite_3 = 1/math.sqrt(variance_3*binom_3)
            scaling_3 = 90*math.sqrt(90)/math.sqrt(2*binom_3)
            for k in numba.prange(N):
                if choice_scaling == 1: #finite variance scaling
                    A[k,0] = scaling_finite_2*X[k,0]
                    A[k,1] = scaling_finite_3*X[k,1]
                    A[k,2] = (1/np.sqrt(2))*(A[k,0]+A[k,1])
                    if banded_model == 0:    
                        A2[k,0] = scaling_finite_2*X2[k,0]
                        A2[k,1] = scaling_finite_3*X2[k,1]
                        A2[k,2] = (1/np.sqrt(2))*(A2[k,0]+A2[k,1])
                        A3[k,0] = scaling_finite_2*X3[k,0]
                        A3[k,1] = scaling_finite_3*X3[k,1]
                        A3[k,2] = (1/np.sqrt(2))*(A3[k,0]+A3[k,1])
                    if banded_model == 3:
                        A4[k,0] = scaling_finite_2*X4[k,0]
                        A4[k,1] = scaling_finite_3*X4[k,1]
                        A4[k,2] = (1/np.sqrt(2))*(A4[k,0]+A4[k,1])
                else : #theoretical/asymptotic scaling
                    A[k,0] = scaling_2*X[k,0]
                    A[k,1] = scaling_3*X[k,1]
                    A[k,2] = (1/np.sqrt(2))*(A[k,0]+A[k,1])
                    if banded_model == 0:    
                        A2[k,0] = scaling_2*X2[k,0]
                        A2[k,1] = scaling_3*X2[k,1]
                        A2[k,2] = (1/np.sqrt(2))*(A2[k,0]+A2[k,1])
                        A3[k,0] = scaling_2*X3[k,0]
                        A3[k,1] = scaling_3*X3[k,1]
                        A3[k,2] = (1/np.sqrt(2))*(A3[k,0]+A3[k,1])
                    if banded_model == 3:
                        A4[k,0] = scaling_2*X4[k,0]
                        A4[k,1] = scaling_3*X4[k,1]
                        A4[k,2] = (1/np.sqrt(2))*(A4[k,0]+A4[k,1])
            if banded_model == 0:
                pd.DataFrame(A).to_csv("Banded_Statistics_Band_23{}.csv".format((N,n,d,h_cov,h_1,h_2,h_3,rho,choice_scaling)), header=False, index=False)
                pd.DataFrame(A2).to_csv("Banded_Statistics_Band_Root_23{}.csv".format((N,n,d,h_cov,h_1,h_2,h_3,rho,choice_scaling)), header=False, index=False)
                pd.DataFrame(A3).to_csv("Banded_Statistics_Band_InvRoot_23{}.csv".format((N,n,d,h_cov,h_1,h_2,h_3,rho,choice_scaling)), header=False, index=False)
            elif banded_model==1:
                pd.DataFrame(A).to_csv("Banded_Statistics_AR(1)_23{}.csv".format((N,n,d,h_1,h_2,h_3,rho,choice_scaling)), header=False, index=False)
            elif banded_model==2:
                pd.DataFrame(A).to_csv("Banded_Statistics_Block_23{}.csv".format((N,n,d,h_1,h_2,h_3,rho,choice_scaling)), header=False, index=False)   
            elif banded_model ==3:
                pd.DataFrame(A4).to_csv("Banded_Statistics_Nonlinear_23{}.csv".format((N,n,d,h_1,h_2,h_3,choice_scaling)), header=False, index=False)   
    return 'done!'
 
def total_power_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling):
    X=np.empty((2,3,4))
    X2=np.empty((2,3,4))
    mean = 1
    for j in range(3):
        for n in [60,100]:
            for d in [50,100,200,400]:
                if n==60:
                    n2=0
                elif n==100:
                    n2=1
                if d==50:
                    d2=0
                elif d==100:
                    d2=1
                elif d==200:
                    d2=2
                elif d==400:
                    d2=3
                if banded_model == 0:
                    A=pd.read_csv("Banded_Statistics_Band_23{}.csv".format((N,n,d,h_cov,h_1,h_2,h_3,rho,choice_scaling)),header=None)
                    A2=pd.read_csv("Banded_Statistics_Band_Root_23{}.csv".format((N,n,d,h_cov,h_1,h_2,h_3,rho,choice_scaling)),header=None)
                    A3=pd.read_csv("Banded_Statistics_Band_InvRoot_23{}.csv".format((N,n,d,h_cov,h_1,h_2,h_3,rho,choice_scaling)),header=None)
                elif banded_model==1:
                    A=pd.read_csv("Banded_Statistics_AR(1)_23{}.csv".format((N,n,d,h_1,h_2,h_3,rho,choice_scaling)),header=None)
                elif banded_model==2:
                    A=pd.read_csv("Banded_Statistics_Block_23{}.csv".format((N,n,d,h_1,h_2,h_3,rho,choice_scaling)),header=None)                    
                elif banded_model==3:
                     A=pd.read_csv("Banded_Statistics_Nonlinear_23{}.csv".format((N,n,d,h_1,h_2,h_3,choice_scaling)),header=None)                                   
                Y=A.values
                X[n2,j,d2]=Gaussian_power_for_each_columns_23_new(Y)[0,j]
                if banded_model == 0:
                    Y2=A2.values
                    X2[n2,j,d2]=Gaussian_power_for_each_columns_23_new(Y2)[0,j] 
                    Y3=A3.values
                    X3[n2,j,d2]=Gaussian_power_for_each_columns_23_new(Y3)[0,j]  
    A_1=X[0,:,:]
    B=X[1,:,:]
    out = np.concatenate((A_1,B),axis=0)
    if banded_model == 0:
        A2_1=X2[0,:,:]
        B2=X2[1,:,:]
        out2 = np.concatenate((A2_1,B2),axis=0)  
        A3_1=X3[0,:,:]
        B3=X3[1,:,:]
        out3 = np.concatenate((A3_1,B3),axis=0) 
    if banded_model == 0:
        pd.DataFrame(out).to_csv("Tabular_Banded_Statistics_Band_23{}.csv".format((N,h_cov,h_1,h_2,h_3,rho,choice_scaling)), header=False, index=False)
        pd.DataFrame(out2).to_csv("Tabular_Banded_Statistics_Band_Root_23{}.csv".format((N,h_cov,h_1,h_2,h_3,rho,choice_scaling)), header=False, index=False)
        pd.DataFrame(out3).to_csv("Tabular_Banded_Statistics_Band_InvRoot_23{}.csv".format((N,h_cov,h_1,h_2,h_3,rho,choice_scaling)), header=False, index=False)
    elif banded_model==1:
        pd.DataFrame(out).to_csv("Tabular_Banded_Statistics_AR(1)_23{}.csv".format((N,h_1,h_2,h_3,rho,choice_scaling)), header=False, index=False)
    elif banded_model==2:
        pd.DataFrame(out).to_csv("Tabular_Banded_Statistics_Block_23{}.csv".format((N,h_1,h_2,h_3,rho,choice_scaling)), header=False, index=False)        
    elif banded_model==3:
        pd.DataFrame(out).to_csv("Tabular_Banded_Statistics_Block_23{}.csv".format((N,h_1,h_2,h_3,rho,choice_scaling)), header=False, index=False)                
    return 100*out

In [ ]:
## Does not really work... why?
@numba.njit(parallel=True, fastmath=True)
def iterated_banded_rescaled_statistics_23(X,n,d,choice_scaling,h_1,h_2,h_3): #X is N*2 np.array
    N=X.shape[0]
    out=np.empty((N,3))
    cardinal = banded_cardinal_23(d,h_1,h_2,h_3)
    variance_2 = ((n-2)*(n-2)*(n-1)*(8*n+1))/(32400*n*n*(n+1)*(n+1))
    scaling_finite_2 = 1/math.sqrt(variance_2*cardinal[0])
    scaling_2 = 90/math.sqrt(2*cardinal[0])
    variance_3 = (n-2)*(n-1)*(16*n - 96 + 359/n - 269/n**2 - 963/n**3 - 370/n**4)/(5832000*(n+1)**3) 
    scaling_finite_3 = 1/math.sqrt(variance_3*cardinal[1])
    scaling_3 = 90*math.sqrt(90)/math.sqrt(2*cardinal[1])
    for k in numba.prange(N):
        if choice_scaling == 1: #finite variance scaling
            out[k,0] = scaling_finite_2*X[k,0]
            out[k,1] = scaling_finite_3*X[k,1]
            out[k,2] = (1/np.sqrt(2))*(out[k,0]+out[k,1])
        else : #theoretical/asymptotic scaling
            out[k,0] = scaling_2*X[k,0]
            out[k,1] = scaling_3*X[k,1]
            out[k,2] = (1/np.sqrt(2))*(out[k,0]+out[k,1])
            
#########################################################################################

#Size: Sample Z with parameters rho=0.3 h_cov=5 h=5 + h=10 and sample W=Z^{1/3}
#Power: Sample Z with parameters rho=0.1 h_cov=20 h=5 + h=10 and sample W=Z^{1/3}

###################

N=500
rho=0.3
h_cov=5
h_1=5
h_2=5
h_3=5
banded_model=0
choice_scaling=0
all_data_banded_stat_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling)
total_power_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling)

###################

N=500
rho=0.1
h_cov=20
h_1=5
h_2=5
h_3=5
banded_model=0
choice_scaling=0
all_data_banded_stat_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling)
total_power_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling)

###################

N=500
rho=0.3
h_cov=5
h_1=10
h_2=10
h_3=10
banded_model=0
choice_scaling=0
all_data_banded_stat_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling)
total_power_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling)

###################

N=500
rho=0.1
h_cov=20
h_1=10
h_2=10
h_3=10
banded_model=0
choice_scaling=0
all_data_banded_stat_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling)
total_power_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling)

###################

N=500
h_1=5
h_2=5
h_3=5
banded_model=3
choice_scaling=0
all_data_banded_stat_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling)
total_power_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling)

###################

N=500
h_1=5
h_2=5
h_3=5
banded_model=3
choice_scaling=0
all_data_banded_stat_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling)
total_power_banded_23(N,h_cov,h_1,h_2,h_3,rho,banded_model,choice_scaling)


In [ ]:
################### Prostate and Healthy Datasets

n = 167 # for Prostate_data
n_h = 157 # for Healthy_data
d = 218
choice_scaling = 0 # 0 is for theoretical scaling and 1 for finite variance version
#h_1 = 50 # bandwidth in [50,180]
#h_2 = 50
#h_3 = 50

Normal_data = npr.normal(0,1,size=(n,d))
Prostate_data = np.loadtxt('data_prostate.txt',delimiter=',')
Healthy_data = np.loadtxt('data_healthy.txt',delimiter=',')

################################################################################################################################################
#X_p = banded_raw_statistics_23(Prostate_data,h_1,h_2,h_3)
#X_h = banded_raw_statistics_23(Healthy_data,h_1,h_2,h_3) 
#pd.DataFrame(X_p).to_csv("Raw_23_Prostate{}.csv".format((h_1,h_2,h_3)), header=False, index=False)
#pd.DataFrame(X_h).to_csv("Raw_23_Healthy{}.csv".format((h_1,h_2,h_3)), header=False, index=False) 

#A_p = banded_rescaled_statistics_23(X_p,n,d,h_1,h_2,h_3,choice_scaling)
#A_h = banded_rescaled_statistics_23(X_h,n_h,d,h_1,h_2,h_3,choice_scaling)                                                                            
#pd.DataFrame(A_p).to_csv("Banded_Statistics_23_Prostate{}.csv".format((h_1,h_2,h_3,choice_scaling)), header=False, index=False)
#pd.DataFrame(A_h).to_csv("Banded_Statistics_23_Healthy{}.csv".format((h_1,h_2,h_3,choice_scaling)), header=False, index=False)
################################################################################################################################################

x=50
y2=180
y3=110
h_grid2=np.arange(x,y2,2)
h_grid3=np.arange(x,y3,2)
out_2p = np.empty(h_grid2.size)
out_2h = np.empty(h_grid2.size)
out_2n = np.empty(h_grid2.size)
out_3p = np.empty(h_grid3.size)
out_3h = np.empty(h_grid3.size)
out_3n = np.empty(h_grid3.size)
out_23p = np.empty((h_grid3.size,3))
out_23h = np.empty((h_grid3.size,3))
out_23n = np.empty((h_grid3.size,3))

# banded_cardinal_23(d,h_1,h_2,h_3)[1] = 0 as soon as h larger than 108 when d is 218

for h in h_grid2:
    if 109>h:
        X_3n = banded_raw_statistics_3(Normal_data,h,h,h)
        X_3p = banded_raw_statistics_3(Prostate_data,h,h,h)
        X_3h = banded_raw_statistics_3(Healthy_data,h,h,h)
        X_23p = banded_raw_statistics_23(Prostate_data,h,h,h)
        X_23h = banded_raw_statistics_23(Healthy_data,h,h,h)
        X_23n = banded_raw_statistics_23(Normal_data,h,h,h)
        out_3p[int((h-x)/2)] = banded_rescaled_statistics_3(X_3p,n,d,h,h,h,choice_scaling)
        #pd.DataFrame(out_3p).to_csv("ProstateBandedStat3_h{}.csv".format((h,choice_scaling)), header=False, index=False)
        out_3h[int((h-x)/2)] = banded_rescaled_statistics_3(X_3h,n_h,d,h,h,h,choice_scaling) 
        #pd.DataFrame(out_3h).to_csv("HealthyBandedStat3_h{}.csv".format((h,choice_scaling)), header=False, index=False)
        out_3n[int((h-x)/2)] = banded_rescaled_statistics_3(X_3n,n_h,d,h,h,h,choice_scaling) 
        out_23p[int((h-x)/2),:] = banded_rescaled_statistics_23(X_23p,n,d,h,h,h,choice_scaling)
        #pd.DataFrame(out_23p).to_csv("ProstateBandedStat23_h{}.csv".format((h,choice_scaling)), header=False, index=False)
        out_23h[int((h-x)/2),:] = banded_rescaled_statistics_23(X_23h,n_h,d,h,h,h,choice_scaling)
        #pd.DataFrame(out_23h).to_csv("HealthyBandedStat23_h{}.csv".format((h,choice_scaling)), header=False, index=False)
        out_23n[int((h-x)/2),:] = banded_rescaled_statistics_23(X_23n,n_h,d,h,h,h,choice_scaling)
    X_2p = banded_raw_statistics_2(Prostate_data,h)
    X_2h = banded_raw_statistics_2(Healthy_data,h)
    X_2n = banded_raw_statistics_2(Normal_data,h)
    out_2p[int((h-x)/2)] = banded_rescaled_statistics_2(X_2p,n,d,h,choice_scaling)
    #pd.DataFrame(out_2p).to_csv("ProstateBandedStat2_h{}.csv".format((h,choice_scaling)), header=False, index=False)
    out_2h[int((h-x)/2)] = banded_rescaled_statistics_2(X_2h,n_h,d,h,choice_scaling) 
    #pd.DataFrame(out_2h).to_csv("HealthyBandedStat2_h{}.csv".format((h,choice_scaling)), header=False, index=False)
    out_2n[int((h-x)/2)] = banded_rescaled_statistics_2(X_2n,n_h,d,h,choice_scaling) 

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt    
legend_dict = { 'S2' : 'blue', 'S3' : 'green', 'T3' : 'red' }
patchList = []
for key in legend_dict:
        data_key = mpatches.Patch(color=legend_dict[key], label=key)
        patchList.append(data_key)
plt.legend(handles=patchList)
plt.plot(np.arange(x,y2,2),out_2p,color='blue')
plt.plot(np.arange(x,y3,2),out_3p,color='green')
plt.plot(np.arange(x,y3,2),out_23p[:,2],color='red')
plt.title("Prostate cancer group")
plt.ylabel("Test statistic")
plt.xlabel("Bandwidth")
plt.savefig('Prostate.png', bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt    
legend_dict = { 'S2' : 'blue', 'S3' : 'green', 'T3' : 'red' }
patchList = []
for key in legend_dict:
        data_key = mpatches.Patch(color=legend_dict[key], label=key)
        patchList.append(data_key)
plt.legend(handles=patchList)
plt.plot(np.arange(x,y2,2),out_2h,color='blue')
plt.plot(np.arange(x,y3,2),out_3h,color='green')
plt.plot(np.arange(x,y3,2),out_23h[:,2],color='red')
plt.title("Healthy group")
plt.ylabel("Test statistic")
plt.xlabel("Bandwidth")
plt.savefig('Healthy.png', bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt    
legend_dict = { 'S2' : 'blue', 'S3' : 'green', 'T3' : 'red' }
patchList = []
for key in legend_dict:
        data_key = mpatches.Patch(color=legend_dict[key], label=key)
        patchList.append(data_key)
plt.legend(handles=patchList)
plt.plot(np.arange(x,y2,2),out_2n,color='blue')
plt.plot(np.arange(x,y3,2),out_3n,color='green')
plt.plot(np.arange(x,y3,2),out_23n[:,2],color='red')
plt.title("Independent Gaussian model")
plt.ylabel("Test statistic")
plt.xlabel("Bandwidth")
plt.savefig('Independent Gaussian model.png', bbox_inches='tight')
plt.show()